In [213]:
import pandas as pd
pandas.set_option('display.max_columns', 10)
pandas.set_option('display.width', 400)

In [ ]:
#read from local file

In [219]:
df = pd.read_excel('C:/Users/brian/OneDrive/Desktop/Rogers-DA/SN Now Churn Data.xlsx')

In [220]:
df.head()

,GUID,StartDate,EndDate,YearMonthChurn,Churn,...,Poker,Basketball,Baseball,Hockey,SportsnetContent
0,0001f31f-d990-4bd7-9d0a-eeb877397b2c,2018-04-11,2018-05-11,20184.0,1,...,0,1,0,2,6
1,000262b9-b373-4730-ab51-01f93bc5655f,2018-06-16,2018-07-16,20186.0,1,...,0,0,1,0,0
2,000616af-15b0-4eaf-b88f-03d40f7d70f4,2016-09-24,2016-10-24,20169.0,1,...,0,0,0,0,0
3,00069260-e6df-444f-8513-a489ae2fc4c6,2016-11-09,2016-12-09,201611.0,1,...,0,0,0,3,0
4,0006c296-300b-4bf8-9181-889033dae996,2017-08-19,2017-09-19,NaN,0,...,0,0,0,0,0


In [80]:
#YearMonthChurn Column is not in the right format
#df['YearMonthChurn'].dtype was float64

dtype('<M8[ns]')

In [221]:
#Convert YearMonthChurn to date
df['YearMonthChurn']= pd.to_datetime(df['YearMonthChurn'], format='%Y%m')
df['YearMonthChurn'] = df['YearMonthChurn'].dt.strftime('%Y-%m')
df.head()

,GUID,StartDate,EndDate,YearMonthChurn,Churn,...,Poker,Basketball,Baseball,Hockey,SportsnetContent
0,0001f31f-d990-4bd7-9d0a-eeb877397b2c,2018-04-11,2018-05-11,2018-04,1,...,0,1,0,2,6
1,000262b9-b373-4730-ab51-01f93bc5655f,2018-06-16,2018-07-16,2018-06,1,...,0,0,1,0,0
2,000616af-15b0-4eaf-b88f-03d40f7d70f4,2016-09-24,2016-10-24,2016-09,1,...,0,0,0,0,0
3,00069260-e6df-444f-8513-a489ae2fc4c6,2016-11-09,2016-12-09,2016-11,1,...,0,0,0,3,0
4,0006c296-300b-4bf8-9181-889033dae996,2017-08-19,2017-09-19,NaN,0,...,0,0,0,0,0


In [222]:
df.describe()

,Churn,TotalStreams,Soccer,Poker,Basketball,Baseball,Hockey,SportsnetContent
count,264122.000000,264122.000000,264122.000000,264122.000000,264122.000000,264122.000000,264122.000000,264122.000000
mean,0.186493,3.012979,0.230378,0.644785,0.296969,0.343667,0.893087,0.604092
std,0.389505,7.942085,1.160791,2.937400,1.489738,1.679017,2.794850,2.228061
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,222.000000,56.000000,63.000000,52.000000,72.000000,86.000000,82.000000


In [279]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import pandas_profiling
import plotly.offline as po
import plotly.graph_objs as go
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
%matplotlib inline
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
#pandas_profiling.ProfileReport(df)

In [224]:
#Handle Missing Values
df['YearMonthChurn'].fillna(0, inplace = True)
df.head()

,GUID,StartDate,EndDate,YearMonthChurn,Churn,...,Poker,Basketball,Baseball,Hockey,SportsnetContent
0,0001f31f-d990-4bd7-9d0a-eeb877397b2c,2018-04-11,2018-05-11,2018-04,1,...,0,1,0,2,6
1,000262b9-b373-4730-ab51-01f93bc5655f,2018-06-16,2018-07-16,2018-06,1,...,0,0,1,0,0
2,000616af-15b0-4eaf-b88f-03d40f7d70f4,2016-09-24,2016-10-24,2016-09,1,...,0,0,0,0,0
3,00069260-e6df-444f-8513-a489ae2fc4c6,2016-11-09,2016-12-09,2016-11,1,...,0,0,0,3,0
4,0006c296-300b-4bf8-9181-889033dae996,2017-08-19,2017-09-19,0,0,...,0,0,0,0,0


In [259]:
df['StartYear']=df['StartDate'].dt.year
df['StartMonth']=df['StartDate'].dt.month
df['week_of_month'] = np.floor((df['StartDate'].dt.day - 1) / 7) + 1
# Extract the day of the week from the date column and map it to a number
df['day_of_week'] = df['StartDate'].dt.dayofweek.map({0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'})
df['day_of_week_num'] = df['StartDate'].dt.dayofweek
df=df.sort_values(by=['GUID','StartDate'])
# Group the dataframe by UserID and calculate the difference between consecutive months
df['MonthDiff'] = df.groupby('GUID')['StartMonth'].diff()
# Create a new column 'ConsecutiveMonths' with the cumulative sum of MonthDiff where MonthDiff == 1
df['ConsecutiveMonths'] = (df['MonthDiff'] == 1).cumsum() + 1
df.head()

,GUID,StartDate,EndDate,YearMonthChurn,Churn,...,day_of_week,ConsecutiveMonths,StartMonth,day_of_week_num,MonthDiff
0,0001f31f-d990-4bd7-9d0a-eeb877397b2c,2018-04-11,2018-05-11,2018-04,1,...,Wednesday,1,4,2,NaN
1,000262b9-b373-4730-ab51-01f93bc5655f,2018-06-16,2018-07-16,2018-06,1,...,Saturday,1,6,5,NaN
66,00034bc0-1e3f-4eb3-a9c6-d3790b76124f,2016-10-20,2016-11-20,2016-10,1,...,Thursday,1,10,3,NaN
2,000616af-15b0-4eaf-b88f-03d40f7d70f4,2016-09-24,2016-10-24,2016-09,1,...,Saturday,1,9,5,NaN
67,00069260-e6df-444f-8513-a489ae2fc4c6,2016-10-09,2016-11-09,0,0,...,Sunday,1,10,6,NaN


In [270]:
# Reset the ConsecutiveMonths for each new UserID
#df['ConsecutiveMonths'] = df.groupby('GUID')['ConsecutiveMonths'].cumsum()
#df['ConsecutiveMonths'] = df.groupby('GUID').cumcount() + 1

# Drop the Month and MonthDiff columns
df = df.drop(['MonthDiff'], axis=1)
df.head(30)

,GUID,StartDate,EndDate,YearMonthChurn,Churn,...,week_of_month,day_of_week,ConsecutiveMonths,StartMonth,day_of_week_num
0,0001f31f-d990-4bd7-9d0a-eeb877397b2c,2018-04-11,2018-05-11,2018-04,1,...,2.0,Wednesday,1,4,2
1,000262b9-b373-4730-ab51-01f93bc5655f,2018-06-16,2018-07-16,2018-06,1,...,3.0,Saturday,1,6,5
66,00034bc0-1e3f-4eb3-a9c6-d3790b76124f,2016-10-20,2016-11-20,2016-10,1,...,3.0,Thursday,1,10,3
2,000616af-15b0-4eaf-b88f-03d40f7d70f4,2016-09-24,2016-10-24,2016-09,1,...,4.0,Saturday,1,9,5
67,00069260-e6df-444f-8513-a489ae2fc4c6,2016-10-09,2016-11-09,0,0,...,2.0,Sunday,1,10,6
3,00069260-e6df-444f-8513-a489ae2fc4c6,2016-11-09,2016-12-09,2016-11,1,...,2.0,Wednesday,2,11,2
4,0006c296-300b-4bf8-9181-889033dae996,2017-08-19,2017-09-19,0,0,...,3.0,Saturday,2,8,5
5,0006c296-300b-4bf8-9181-889033dae996,2017-09-19,2017-10-19,0,0,...,3.0,Tuesday,3,9,1
6,0006c296-300b-4bf8-9181-889033dae996,2017-10-19,2017-11-19,0,0,...,3.0,Thursday,4,10,3
68,0006c296-300b-4bf8-9181-889033dae996,2017-11-19,2017-12-19,0,0,...,3.0,Sunday,5,11,6


In [269]:
# Find the columns with missing values
#cols_with_na = df.columns[df.isna().sum() > 0].tolist()
#print('Columns with missing values:', cols_with_na)

Columns with missing values: ['MonthDiff']


In [272]:
# Split the data into features and target variables
X = df.drop(['Churn','GUID','StartDate','EndDate','YearMonthChurn','day_of_week'], axis=1)
#'GUID','EndDate','YearMonthChurn'
y = df['Churn']
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.34, random_state=42)
# Train a random forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
# Make predictions on the test set
y_pred = clf.predict(X_test)
# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)

Accuracy: 0.7415536402307298
Precision: 0.25141659551346734
Recall: 0.1927517257795763
F1-score: 0.21820999090511


In [282]:
d = {'Features': X_train.columns, 'Feature Importance': clf.feature_importances_}
df = pd.DataFrame(d)
df_sorted = df.sort_values(by='Feature Importance', ascending = True)
df_sorted
df_sorted.style.background_gradient(cmap='Blues')

,Features,Feature Importance
3,Basketball,0.008739
1,Soccer,0.008799
4,Baseball,0.011183
2,Poker,0.011654
6,SportsnetContent,0.014813
5,Hockey,0.015229
7,StartYear,0.015707
0,TotalStreams,0.028485
8,week_of_month,0.039171
11,day_of_week_num,0.052968


In [283]:
fig = px.bar(x=df_sorted['Feature Importance'], y=df_sorted['Features'], color_continuous_scale=px.colors.sequential.Blues,
             title='<b>Feature Importance Based on Random Forest<b>', text_auto='.4f', color=df_sorted['Feature Importance'])

fig.update_traces(marker=dict(line=dict(color='black', width=2)))
fig.update_layout({'yaxis': {'title':'Features'}, 'xaxis': {'title':'Feature Importance'}})

iplot(fig)